# What is Selenium?

## 1

What environment do we use?

selenium docker?

See [What is Docker?](What%20is%20Docker.ipynb)



In [6]:
var importer = require('../Core');
var fs = require('fs');

// add some run commands to the bash script
function bashToRun(code) {
    return code.split('\n').filter(n => n.trim().length > 0).map(l => 'RUN ' + l.trim())
    .join('\n').replace(/\\\s*\nRUN\s*/ig, '\\\n ');
}

// create a selenium Dockerfile with a vnc connection
function seleniumDocker(outputFile) {
    const r = importer.interpret([
        'run selenium inside docker',
        // add some extra services
        'linux dev tools',
        'vnc html',
        'vnc docker'
    ]);
    // save the Dockerfile
    fs.writeFileSync(outputFile, [
        r[0].code,
        // convert some results to Docker RUN commands instead of bash
        bashToRun(r[1].code),
        bashToRun(r[2].code),
        r[3].code
    ].join('\n'));
    return r;
};
module.exports = seleniumDocker;


[Function: seleniumDocker]

How to set up selenium server?

In [7]:
var importer = require('../Core');
var path = require('path');
var fs = require('fs');
var execCmd = importer.import('spawn child process');

var DOWNLOAD_DIR = path.join(process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE, 'Downloads');
var PROFILE_DIR = path.join(process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE, '.defaultProfile');
var DOCKERFILE = path.resolve(path.join(__dirname, 'Dockerfile'));

function getSeleniumServer(name = 'act-selenium') {
    try {
        fs.mkdirSync(DOWNLOAD_DIR);
    } catch (err) {
        if (err.code != 'EEXIST') {
            throw err;
        }
    }
    try {
        fs.mkdirSync(PROFILE_DIR);
    } catch (err) {
        if (err.code != 'EEXIST') {
            throw err;
        }
    }
    try {
        fs.unlinkSync(path.join(PROFILE_DIR, 'SingletonLock'));
    } catch (err) {
        if (err.code != 'ENOENT') {
            throw err;
        }
    }
    seleniumDocker(DOCKERFILE);
    return execCmd('docker ps -a')
        .then(r => {
            if (r[0].indexOf(name) > -1) {
                return execCmd('docker stop ' + name)
                    .then(r => new Promise(resolve =>
                        setTimeout(() => resolve(r), 1000)))
                    .then(() => execCmd('docker rm ' + name));
            }
        })
        .then(() => new Promise(resolve =>
            setTimeout(() => resolve(), 1000)))
        .then(() => {
            var build = 'docker build -t ' + name + ' "'
                + path.dirname(DOCKERFILE) + '"\n'
                + 'docker run --shm-size=3g -d '
                + '--name ' + name + ' '
                + '-p 8888:8888 '
                + '-p 6080:6080 '
                + '-p 5900:5900 '
                + '-p 4444:4444 '
                + '-p 4200:4200 '
                + '-p 3000:3000 '
                // TODO: add profile dir back in when permissions works on windows
                + '-v "' + DOWNLOAD_DIR + '":/data/downloads '
                + name + '\n';
            return execCmd(build)
        })
        .then(r => new Promise(resolve => setTimeout(() => resolve(r), 6000)))
        .catch(e => console.log(e))
};
module.exports = getSeleniumServer;
getSeleniumServer;



[Function: getSeleniumServer]

In [2]:
var importer = require('../Core');
var {getSeleniumServer} = importer.import([
    'What is Selenium.ipynb[what is selenium]',
    'set up selenium server'
]);

$$.async();
getSeleniumServer()
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e));



CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES

d1d66d7a8f27086824f8e845245aef020880de5e43351a505dd0b1e90c64bd0d



Step 1/25 : FROM selenium/standalone-chrome-debug

 ---> e5d173515f75

Step 2/25 : EXPOSE 4200

 ---> Using cache
 ---> de4aacbca0e7
Step 3/25 : EXPOSE 4444

 ---> Using cache
 ---> 9824efc0efee
Step 4/25 : EXPOSE 3000

 ---> Using cache
 ---> 90f8baf644e6
Step 5/25 : ENV DBUS_SESSION_BUS_ADDRESS /dev/null

 ---> Using cache
 ---> e623a9c291c0

Step 6/25 : ENV CHROME_USER_DATA_DIR /usr/profile

 ---> Using cache
 ---> a3dfb094f7c7

Step 7/25 : WORKDIR /home/seluser

 ---> Using cache
 ---> 0db91782a4cd

Step 8/25 : USER root

 ---> Using cache
 ---> 8d357dc0e08d

Step 9/25 : RUN mkdir /usr/profile

 ---> Using cache
 ---> 3bd85a5247b3
Step 10/25 : RUN mkdir /usr/downloads

 ---> Using cache
 ---> 6f8e1705c264
Step 11/25 : RUN chown seluser:seluser -R /usr/profile

 ---> Using cache
 ---> 4c9723

[ 'Sending build context to Docker daemon  1.146MB\r\r\nStep 1/25 : FROM selenium/standalone-chrome-debug\n ---> e5d173515f75\nStep 2/25 : EXPOSE 4200\n ---> Using cache\n ---> de4aacbca0e7\nStep 3/25 : EXPOSE 4444\n ---> Using cache\n ---> 9824efc0efee\nStep 4/25 : EXPOSE 3000\n ---> Using cache\n ---> 90f8baf644e6\nStep 5/25 : ENV DBUS_SESSION_BUS_ADDRESS /dev/null\n ---> Using cache\n ---> e623a9c291c0\nStep 6/25 : ENV CHROME_USER_DATA_DIR /usr/profile\n ---> Using cache\n ---> a3dfb094f7c7\nStep 7/25 : WORKDIR /home/seluser\n ---> Using cache\n ---> 0db91782a4cd\nStep 8/25 : USER root\n ---> Using cache\n ---> 8d357dc0e08d\nStep 9/25 : RUN mkdir /usr/profile\n ---> Using cache\n ---> 3bd85a5247b3\nStep 10/25 : RUN mkdir /usr/downloads\n ---> Using cache\n ---> 6f8e1705c264\nStep 11/25 : RUN chown seluser:seluser -R /usr/profile\n ---> Using cache\n ---> 4c97231d1909\nStep 12/25 : RUN chown seluser:seluser -R /usr/downloads\n ---> Using cache\n ---> 0a00241ced53\nStep 13/25 : RUN ch

# 2

How to connect to VNC through the web browser?

[Click here to open](http://localhost:6080/vnc.html?password=secret&host=localhost&port=6080&autoconnect=true&resize=downscale&view_only=true&reconnect=true) in a separate browser


In [1]:
function urlEncode(obj) {
    return Object.keys(obj).map(k => encodeURIComponent(k) + '=' + encodeURIComponent(obj[k])).join('&');
}

var vncIframe = (options = {
    password: 'secret',
    host: 'localhost',
    port: 6080,
    autoconnect: true,
    resize: 'downscale',
    view_only: true,
    reconnect: true
}) => `
<div style="display:block; width:100%; padding-bottom:69%;position:relative;">
<iframe id="vnc"
style="position:absolute;top:0;right:0;bottom:0;left:0;width:100%;height:100%;border:0;" 
src="//local:6080/vnc.html?${urlEncode(options)}"></iframe></div>
`;
module.exports = vncIframe;
vncIframe;



[Function: vncIframe]

In [2]:
$$.mime({'text/html': vncIframe()});


## 3

How to run a selenium cell on the Docker machine?



In [11]:
var importer = require('../Core');

function runSeleniumCell(search, newWindow = true) {
    var createWebdriverClient = importer.import('webdriver client');
    var client = createWebdriverClient();
    var ctx = {client};
    var {
        getCredentials,
        getAllXPath,
        getAllUntil,
        resizeWindow,
        onlyOneWindow,
        getAllSessionUrls,
        clickSpa,
    } = importer.import([
        'decrypt password',
        'all elements xpath',
        'get all elements until',
        'resize selenium window',
        'only one window',
        'get all session urls',
        'click spa link',
    ], ctx);
    var formUtilities = importer.import('form utilities', ctx);
    return client
        .getActiveSessions()
        .then(() => (newWindow
                     ? client
                     .then(() => onlyOneWindow())
                     .then(() => getAllSessionUrls())
                     : client))
        .then(() => {
            if(typeof search === 'undefined') {
                return client;
            }
            return importer.import(search, Object.assign(ctx, {
                client: client,
                browser: client,
                getCredentials,
                getAllXPath,
                getAllUntil,
                resizeWindow,
                clickSpa,
            }, formUtilities))
        })
};
module.exports = runSeleniumCell;


[Function: runSeleniumCell]

In [ ]:
var importer = require('../Core');
var runSeleniumCell = importer.import('selenium cell');

$$.async()
runSeleniumCell('test docker selenium')
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e));


Initializing webdriver on localhost
[ 1510335301592, '80302827-6a4d-461e-80a1-bfc09636544a' ]
[14:00:50]  COMMAND	GET 	 "/wd/hub/session/80302827-6a4d-461e-80a1-bfc09636544a/window_handle"
[ 1510335301652, '61a7d2c9-2cad-45a4-b14a-1e14e0110385' ]
[14:01:31]  COMMAND	GET 	 "/wd/hub/session/61a7d2c9-2cad-45a4-b14a-1e14e0110385/window_handle"
[ 1510335301671, '45060462-a7c8-4412-8ad2-d86bd346bd1c' ]
[14:02:11]  COMMAND	GET 	 "/wd/hub/session/45060462-a7c8-4412-8ad2-d86bd346bd1c/window_handle"
[ 1510335301696, '2ceb787b-3923-4d3b-b179-eff941c50b3e' ]
[14:02:42]  COMMAND	GET 	 "/wd/hub/session/2ceb787b-3923-4d3b-b179-eff941c50b3e/window_handle"
[ 1510335301731, 'eb35a1f1-c85b-400b-aeea-355389966f56' ]
[14:02:55]  COMMAND	GET 	 "/wd/hub/session/eb35a1f1-c85b-400b-aeea-355389966f56/window_handle"
[ 1510335301745, '317dede0-4d5b-42cd-83f7-361d9e844038' ]
[14:03:35]  COMMAND	GET 	 "/wd/hub/session/317dede0-4d5b-42cd-83f7-361d9e844038/window_handle"
[ 1510335309541, 'f4fbb8ba-6e67-410c-999e-0446

Test docker selenium?

In [ ]:
function testLive() {
    return client
        .url('https://purchasesprint.actops.com')
        .click('[href*="/auth"], [routerlink*="/auth"]');
}
module.exports = testLive();




TODO:

Auto fill fields in a live browser using the 

Transfer login state to current browser?

